In [8]:
%load_ext autoreload
%autoreload 2

from experiments.all import train_and_evaluate
from tsm.configuration import TSMConfigParser
import docker
import logging

logging.basicConfig(level=logging.INFO)

# load configuration
config = TSMConfigParser()
config.read("/home/pablo/tesis/tsm-pipeline/code/configs/pablo-bestia.cfg")
config.check()
docker_client = docker.from_env()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
from scripts.docker import read_dbs_dataset
from numpy.random import RandomState

sql_dbs = read_dbs_dataset("/home/pablo/tesis/tsm-pipeline/experiments/datasets/sql_1.txt")
# Using a fixed random state to have reproducible results
rand = RandomState(seed=5)

print(len(sql_dbs))

42


Testing the sql module works

In [5]:
train_sample = rand.choice(sql_dbs, size=5)
test_sample = rand.choice(sql_dbs, size=3)

In [ ]:
scores = train_and_evaluate(
            config=config,
            results_dir="/home/pablo/dev-results/sql-test",
            query_type="sql",
            train=train_sample.tolist(),
            test=test_sample.tolist(),
            docker_client=docker_client,
            skip=["train"] # skip training since it has already ran
        )
scores

## Corrida para tesis

In [11]:
from sklearn.model_selection import KFold
# fixing random state to make this repeatable
kf = KFold(n_splits=5)
folds = list(kf.split(sql_dbs))

In [12]:
folds

[(array([ 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25,
         26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41]),
  array([0, 1, 2, 3, 4, 5, 6, 7, 8])),
 (array([ 0,  1,  2,  3,  4,  5,  6,  7,  8, 18, 19, 20, 21, 22, 23, 24, 25,
         26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41]),
  array([ 9, 10, 11, 12, 13, 14, 15, 16, 17])),
 (array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
         17, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41]),
  array([18, 19, 20, 21, 22, 23, 24, 25])),
 (array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
         17, 18, 19, 20, 21, 22, 23, 24, 25, 34, 35, 36, 37, 38, 39, 40, 41]),
  array([26, 27, 28, 29, 30, 31, 32, 33])),
 (array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
         17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33]),
  array([34, 35, 36, 37, 38, 39, 40, 41]))]

In [14]:
import numpy as np
base_results_dir = "/home/pablo/dev-results/sql-5fold-"
for i, (train_index, test_index) in enumerate(folds):
    try:
        print("Fold %d" % (i))
        res_dir = "%s%d" % (base_results_dir, i)
        train_split = np.take(sql_dbs, train_index)
        test_split = np.take(sql_dbs, test_index)

        scores = train_and_evaluate(
            config=config,
            results_dir=res_dir,
            query_type="sql",
            train=train_split.tolist(),
            test=test_split.tolist(),
            docker_client=docker_client,
        )
        print(scores)
    except Exception as e:
        print(f'fold {i} failed fatally. Please retry!')
        print(e)
        continue
print("done")

[INFO	2023-07-23 14:52:09,306] experiments.all	train_and_evaluate with train size 33, test size 9
[INFO	2023-07-23 14:52:09,307] experiments.all	Running training
[INFO	2023-07-23 14:52:09,307] scripts.docker	creating /home/pablo/dev-results/sql-5fold-0 dir. Omitting error if already existing
[INFO	2023-07-23 14:52:09,307] scripts.docker	creating /tmp/log dir. Omitting error if already existing


Fold 0


[INFO	2023-07-23 14:52:09,586] scripts.docker	running at container b9a91c0330698bcc950484d07d45657ecb53f505d2eb4a563c7274b2c639e950. Use `docker logs b9a91c0330698bcc950484d07d45657ecb53f505d2eb4a563c7274b2c639e950 --tail 10 --follow` to follow progress
[INFO	2023-07-23 15:00:18,753] scripts.docker	running combine scores
[INFO	2023-07-23 15:00:18,754] misc.combinescores	globbing folder: /home/pablo/dev-results/sql-5fold-0/*/reprScores.txt
[INFO	2023-07-23 15:00:18,755] misc.combinescores	working on 22 reprScores files
[INFO	2023-07-23 15:00:18,759] experiments.all	Running worse evaluation
[INFO	2023-07-23 15:00:18,759] scripts.evaluate	creating /home/pablo/dev-results/sql-5fold-0/worse dir. Omitting error if already existing
[INFO	2023-07-23 15:00:18,938] scripts.evaluate	Starting processing with 4 processes
[INFO	2023-07-23 15:00:18,959] scripts.evaluate	Evaluating TheKingdomDev - Burger
[INFO	2023-07-23 15:00:18,959] scripts.evaluate	Evaluating OWASP - samm
[INFO	2023-07-23 15:00:18,

   precision  recall  accuracy  alerts to recover (atr)  alerts recovered  \
0        0.0     0.0       0.0                    13599                 0   

   suprious alerts  projects with atr  avg atr per proj  
0                2                  8          1699.875  
Fold 1


[INFO	2023-07-23 15:01:02,710] scripts.docker	running at container 1646757a7b1e490ba232be8898c4a7ae04da08ee4a17ceba3428bac2e0e96e40. Use `docker logs 1646757a7b1e490ba232be8898c4a7ae04da08ee4a17ceba3428bac2e0e96e40 --tail 10 --follow` to follow progress
[INFO	2023-07-23 15:07:39,007] scripts.docker	running combine scores
[INFO	2023-07-23 15:07:39,007] misc.combinescores	globbing folder: /home/pablo/dev-results/sql-5fold-1/*/reprScores.txt
[INFO	2023-07-23 15:07:39,008] misc.combinescores	working on 26 reprScores files
[INFO	2023-07-23 15:07:39,012] experiments.all	Running worse evaluation
[INFO	2023-07-23 15:07:39,012] scripts.evaluate	creating /home/pablo/dev-results/sql-5fold-1/worse dir. Omitting error if already existing
[INFO	2023-07-23 15:07:39,194] scripts.evaluate	Starting processing with 4 processes
[INFO	2023-07-23 15:07:39,211] scripts.evaluate	Evaluating cbrannon - burger
[INFO	2023-07-23 15:07:39,210] scripts.evaluate	Evaluating arukpupatee - what2eat
[INFO	2023-07-23 15:0

...

   precision  recall  accuracy  alerts to recover (atr)  alerts recovered  \
0        0.0     0.0       0.0                       21                 0   

   suprious alerts  projects with atr  avg atr per proj  
0               54                  5               4.2  
Fold 3


[INFO	2023-07-23 15:15:59,022] scripts.docker	running at container 82beb1b0eb5441fd835347f03fac7ff4575d858f0cfc63f10ecf4b358effdd1b. Use `docker logs 82beb1b0eb5441fd835347f03fac7ff4575d858f0cfc63f10ecf4b358effdd1b --tail 10 --follow` to follow progress
[INFO	2023-07-23 15:22:48,674] scripts.docker	running combine scores
[INFO	2023-07-23 15:22:48,674] misc.combinescores	globbing folder: /home/pablo/dev-results/sql-5fold-3/*/reprScores.txt
[INFO	2023-07-23 15:22:48,675] misc.combinescores	working on 25 reprScores files
[INFO	2023-07-23 15:22:48,680] experiments.all	Running worse evaluation
[INFO	2023-07-23 15:22:48,680] scripts.evaluate	creating /home/pablo/dev-results/sql-5fold-3/worse dir. Omitting error if already existing
[INFO	2023-07-23 15:22:48,859] scripts.evaluate	Starting processing with 4 processes
[INFO	2023-07-23 15:22:48,880] scripts.evaluate	Evaluating kwuellner - Burger
[INFO	2023-07-23 15:22:48,880] scripts.evaluate	Evaluating mheadd - civic-responsibility-api
[INFO	202

   precision  recall  accuracy  alerts to recover (atr)  alerts recovered  \
0        0.0     0.0       0.0                        5                 0   

   suprious alerts  projects with atr  avg atr per proj  
0                4                  3          1.666667  
Fold 4


[INFO	2023-07-23 15:23:23,335] scripts.docker	running at container 0f6e5aa0892d8f63c3e7a3a1a06dbec72f8541cd26b4a8b1052388116183ca8f. Use `docker logs 0f6e5aa0892d8f63c3e7a3a1a06dbec72f8541cd26b4a8b1052388116183ca8f --tail 10 --follow` to follow progress
[INFO	2023-07-23 15:29:10,731] scripts.docker	running combine scores
[INFO	2023-07-23 15:29:10,732] misc.combinescores	globbing folder: /home/pablo/dev-results/sql-5fold-4/*/reprScores.txt
[INFO	2023-07-23 15:29:10,732] misc.combinescores	working on 26 reprScores files
[INFO	2023-07-23 15:29:10,737] experiments.all	Running worse evaluation
[INFO	2023-07-23 15:29:10,737] scripts.evaluate	creating /home/pablo/dev-results/sql-5fold-4/worse dir. Omitting error if already existing
[INFO	2023-07-23 15:29:10,914] scripts.evaluate	Starting processing with 4 processes
[INFO	2023-07-23 15:29:10,936] scripts.evaluate	Evaluating vulhub - vulhub
[INFO	2023-07-23 15:29:10,936] scripts.evaluate	Evaluating startergate - EnglishPresentation
[INFO	2023-0

   precision  recall  accuracy  alerts to recover (atr)  alerts recovered  \
0        0.0     0.0       0.0                        5                 0   

   suprious alerts  projects with atr  avg atr per proj  
0                2                  2               2.5  
done


In [15]:
import pandas as pd
from scripts.calculate_scores import calculate_scores_df

collected_results = []
for i in range(5):
    df = calculate_scores_df("%s%d" % (base_results_dir, i), cleanup_base_dir="tmp")
    collected_results.append(df)

concated_results = pd.concat(collected_results)
concated_results = pd.concat([concated_results, concated_results.apply(['mean'])])
concated_results['name'] = [f'fold {i}' for i in range(5)] + ['mean']
concated_results

,precision,recall,accuracy,alerts to recover (atr),alerts recovered,suprious alerts,projects with atr,avg atr per proj,name
0,0.0,0.0,0.0,13599.0,0.0,2.0,8.0,1699.875000,fold 0
0,0.0,0.0,0.0,22.0,0.0,2.0,7.0,3.142857,fold 1
0,0.0,0.0,0.0,21.0,0.0,54.0,5.0,4.200000,fold 2
0,0.0,0.0,0.0,5.0,0.0,4.0,3.0,1.666667,fold 3
0,0.0,0.0,0.0,5.0,0.0,2.0,2.0,2.500000,fold 4
mean,0.0,0.0,0.0,2730.4,0.0,12.8,5.0,342.276905,mean
